# End-to-End AutoML for Insurance Cross-Sell

## Part 1 - EDA and Data Pre-Processing


### Contents
[Part 1 - Project Overview](#overview)  
[Part 2 - Initial Setup](#setup)  
[Part 3 - Exploratory Data Analysis](#eda)  
[Part 4 - Data Pre-Processing and Transformation](#pre-processing)  
[Part 5 - References](#references) 

___
<a name="overview"></a>
## (1) Project Overview

**The Challenge**  
- Insurance Cross-Sell Model: Predict which existing health insurance customers would be interested in purchasing vehicle insurance as well
- This is a binary classification task

Link: https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction
___
**The Data**  
In this dataset, you are provided variables describing attributes of health insurance applicants. The task is to predict the "Response" variable for each Id in the test set 
___
**id:**	Unique ID for the customer  
**Gender:**	Gender of the customer  
**Age:** Age of the customer  
**Driving_License:** 0 : Customer does not have DL, 1 : Customer already has DL  
**Region_Code:** Unique code for the region of the customer  
**Previously_Insured:**	1 : Customer already has Vehicle Insurance, 0 : Customer doesn't have Vehicle Insurance  
**Vehicle_Age:** Age of the Vehicle  
**Vehicle_Damage:**	1 : Customer got his/her vehicle damaged in the past. 0 : Customer didn't get his/her vehicle damaged in the past.  
**Annual_Premium:**	The amount customer needs to pay as premium in the year  
**PolicySalesChannel:**	Anonymized Code for the channel of outreaching to the customer ie. Different Agents, Over Mail, Over Phone, In Person, etc.  
**Vintage:** Number of Days that the customer has been associated with the company  
**Response**: 1 : Customer is interested, 0 : Customer is not interested  

<a name="setup"></a>
## (2) Initial Setup

### Import dependencies

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn import metrics

import opendatasets as od
import zipfile
import os
import shutil
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

### Download Datasets
To use the Kaggle API, sign up for a Kaggle account at https://www.kaggle.com. Then go to the 'Account' tab of your user profile (https://www.kaggle.com/<username>/account) and select 'Create API Token'. This will trigger the download of kaggle.json, a file containing your API credentials.

In [6]:
# Retrieve data directly from source (using Kaggle API credentials, found in kaggle.json)
od.download("https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction",
           './data/raw/')

100%|██████████| 6.47M/6.47M [00:03<00:00, 2.19MB/s]


In [7]:
# View files in the raw folder
raw_path = './data/raw/health-insurance-cross-sell-prediction/'
os.listdir(raw_path)

['test.csv', 'train.csv', 'sample_submission.csv']

In [8]:
# Move train and test flat files into input folder
for filename in os.listdir(raw_path):
    if 'train' in filename or 'test' in filename:
        shutil.copy(raw_path+filename, './data/input/'+filename)

### Import Data

In [ ]:
# Import input datasets
main_df = pd.read_csv('data/input/train.csv')

# Keep only random 1/3 sample of data (to optimize size for training)
main_df = main_df.sample(frac=0.3, replace=False, random_state=42).reset_index(drop=True)

len(main_df)

Note: We are not using the test.csv dataset because it does not have the target `Response` variable, and we will not be submitting our predictions to Kaggle

### Setup helper functions

In [ ]:
# Display value counts and NaN counts
def show_value_counts(df, column):
    value_count_df = pd.DataFrame(df[column].value_counts().rename_axis(column).reset_index(name='counts'))
    value_count_df['percentage'] = round(100 * (value_count_df['counts'] / len(df)),1)
    return value_count_df

def show_nan_counts(df):
    nan_count_df = pd.DataFrame(df.isna().sum()).sort_values(by=0, ascending = False)
    nan_count_df.columns = ['counts']
    nan_count_df['percentage'] = round(100 * (nan_count_df['counts'] / len(df)),1)
    return nan_count_df

# Display correlation plot as heatmap
def show_heatmap(data, figsize=(12,8), 
                 highest_only=False, 
                 thresold=0.7, # Look at only highly correlated pairs
                 annot=False):
    correlation_matrix = data.corr()
    high_corr = correlation_matrix[np.abs(correlation_matrix)>= thresold]
    plt.figure(figsize=figsize)

    if highest_only:
        sns.heatmap(high_corr, annot=annot, cmap="Greens", 
                    linecolor='black', linewidths=0.1)
    else:
        sns.heatmap(correlation_matrix, annot=annot)


## (3) Exploratory Data Analysis (EDA)

<a name="pre-processing"></a>
## (4) Data Pre-Processing and Transformation